# прогнозирование спроса на новые товары
### иса гадаев павел манякин георгий дудин при поддержке retro goldwyn mayer

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn.model_selection as skr
import statsmodels.formula.api as sm
import statsmodels.api as sm2
import statsmodels.graphics.regressionplots as smg
from sklearn.metrics import r2_score
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

%matplotlib inline

sns.set()

In [2]:
test_df = pd.read_csv("hse-dab-naf-23/test.csv")
pd.set_option('display.max_columns', None)

test_df.head(10)

,product_id,location_id,period_dt,id
0,22939,1162,2019-12-30,16
1,23162,1162,2019-12-30,7867
2,23163,1162,2019-12-16,8667
3,23632,798,2019-12-23,14835
4,23633,1347,2019-12-30,14959
5,24007,764,2019-12-16,24820
6,32646,380,2019-12-16,35688
7,34846,866,2019-12-23,41293
8,34847,764,2019-12-16,41644
9,34847,1034,2019-12-30,41646


In [8]:
train_df = pd.read_csv("hse-dab-naf-23/train.csv", sep=',')

train_df.head(10)

,period_dt,location_id,product_id,demand,PROMO1_FLAG,PROMO2_FLAG,PRICE_REGULAR,PRICE_AFTER_DISC,NUM_CONSULTANT,AUTORIZATION_FLAG,id
0,2019-12-30,425.0,22939.0,NaN,0.0,0.0,3649.0,2736.75,0.0,0.0,1
1,2019-12-30,525.0,22939.0,NaN,0.0,0.0,3649.0,2371.85,0.0,1.0,3
2,2019-12-30,557.0,22939.0,NaN,0.0,0.0,3649.0,2736.75,0.0,1.0,5
3,2019-12-30,562.0,22939.0,NaN,0.0,0.0,3649.0,2736.75,0.0,0.0,7
4,2019-12-30,637.0,22939.0,NaN,0.0,0.0,3649.0,3101.65,0.0,0.0,9
5,2019-12-16,637.0,22939.0,NaN,0.0,0.0,3649.0,3101.65,0.0,0.0,10
6,2019-12-09,637.0,22939.0,NaN,0.0,0.0,3649.0,3101.65,0.0,1.0,11
7,2019-12-30,657.0,22939.0,NaN,0.0,0.0,3649.0,2736.75,0.0,1.0,12
8,2019-12-23,657.0,22939.0,NaN,0.0,0.0,3649.0,2736.75,0.0,1.0,13
9,2019-12-16,764.0,22939.0,NaN,0.0,0.0,3649.0,2371.85,0.0,1.0,14


In [5]:
product_df = pd.read_csv("hse-dab-naf-23/PRODUCT.csv", sep=';')

product_df.head(10)

,PRODUCT_RK,PRODUCT_LVL_RK6,PRODUCT_LVL_RK5,PRODUCT_LVL_RK4,PRODUCT_LVL_RK3,PRODUCT_LVL_RK2,PRODUCT_LVL_RK1,SALES_INTRODUCTION_DT,SALES_DISCONTINUED_DT,VALID_FROM_DTTM,VALID_TO_DTTM,PROCESSED_DTTM,PRODUCT_ADK_hashing,PRODUCT_ID_hashing,PRODUCT_NM_hashing,PRODUCT_DESC_hashing,LANGUAGE_CD_hashing,PRODUCT_LVL_ID6_hashing,PRODUCT_LVL_NM6_hashing,PRODUCT_LVL_DESC6_hashing,PRODUCT_LVL_ID5_hashing,PRODUCT_LVL_NM5_hashing,PRODUCT_LVL_DESC5_hashing,PRODUCT_LVL_ID4_hashing,PRODUCT_LVL_NM4_hashing,PRODUCT_LVL_DESC4_hashing,PRODUCT_LVL_ID3_hashing,PRODUCT_LVL_NM3_hashing,PRODUCT_LVL_DESC3_hashing,PRODUCT_LVL_ID2_hashing,PRODUCT_LVL_NM2_hashing,PRODUCT_LVL_DESC2_hashing,PRODUCT_LVL_ID1_hashing,PRODUCT_LVL_NM1_hashing,PRODUCT_LVL_DESC1_hashing,PRODUCT_ATTRIB1_hashing,PRODUCT_ATTRIB2_hashing,PRODUCT_ATTRIB3_hashing,PRODUCT_ATTRIB4_hashing,PRODUCT_ATTRIB5_hashing,PRODUCT_ATTRIB6_hashing,PRODUCT_ATTRIB7_hashing,PRODUCT_ATTRIB8_hashing,PRODUCT_ATTRIB9_hashing,PRODUCT_ATTRIB10_hashing,PRODUCT_ATTRIB11_hashing,PRODUCT_ATTRIB12_hashing,PRODUCT_ATTRIB13_hashing,PRODUCT_ATTRIB14_hashing,PRODUCT_ATTRIB15_hashing,PRODUCT_ATTRIB16_hashing,PRODUCT_ATTRIB17_hashing,PRODUCT_ATTRIB18_hashing,PRODUCT_ATTRIB19_hashing,PRODUCT_ATTRIB20_hashing,PRODUCT_ATTRIB21_hashing,PRODUCT_ATTRIB22_hashing,PRODUCT_ATTRIB23_hashing,PRODUCT_ATTRIB24_hashing,PRODUCT_ATTRIB25_hashing,PRODUCT_ATTRIB26_hashing,PRODUCT_ATTRIB27_hashing,PRODUCT_ATTRIB28_hashing,PRODUCT_ATTRIB29_hashing,PRODUCT_ATTRIB30_hashing,PRODUCT_ATTRIB31_hashing,PRODUCT_TYPE_CD_hashing
0,11316,10224,10198,113533,10125,9711,1,NaN,NaN,15Mar2018:14:08:14,01Jan5999:00:00:00,03Jan2020:07:01:13,A5C279811C3210EB8CFD7027E51ADAC4,419E9D9BC063B0471F98BA65344206C1,4E055B6F492B9AAD0A791B9D5D3A165E,E5E1FA47D79C1D573A65E2A4D17F6E6E,3A73A94D029959DC71AF0AFCB9F2189E,0DC45A0D604E45ED8BA0A4ED142A386C,6EE6DFA5CFDC795C76C96862BC7F09A4,E5E1FA47D79C1D573A65E2A4D17F6E6E,BC8A69BA32EB41D5455D2EB96F8A12C6,713FF050F1DE07BB666862D98DB97371,E5E1FA47D79C1D573A65E2A4D17F6E6E,0281FFC7E864B8858E7611BB38B9B978,4CBE937C147AE54DF264D9443DBE7335,E5E1FA47D79C1D573A65E2A4D17F6E6E,F1A5375C05FFFD9BB9905FAC97D28B57,95373222989EEE03015ED15345546FAC,E5E1FA47D79C1D573A65E2A4D17F6E6E,7A2CC583A6FFD094E3F6A33CF672D82E,F53A5BABC3AB3A013F7E4648151E89B6,E5E1FA47D79C1D573A65E2A4D17F6E6E,DE3658788669708702FFF3C7D966703C,4A769F24B4FCA27347F42F9EA580C92C,E5E1FA47D79C1D573A65E2A4D17F6E6E,B2581C791DBB8894BFDCFB66B60A17ED,E02A565DBB97BDD239002A471B812C1F,CE6E6B615231B052EF75AEDBED6491A5,FA49AADB69612563EC488572DD6BD92C,DFB3D6F28924754EB60C4E5B39E4D86B,FE26708E98829533D1F2877AC0D06DAC,67442CF09DCF9C0C0C02977AC80A5E52,3797D379D4D05BB733CA37EDFE74A29C,B2581C791DBB8894BFDCFB66B60A17ED,ADB238A5FA27E863EB4F190A2A400A30,5C9AE6F0DD3641D7E0B3DF4936F8EF60,67442CF09DCF9C0C0C02977AC80A5E52,3797D379D4D05BB733CA37EDFE74A29C,B2581C791DBB8894BFDCFB66B60A17ED,B2581C791DBB8894BFDCFB66B60A17ED,0236155B72D4F01753F8D5460D0532E3,FF83472786DB3B25D48ABD61502E1D91,6022A682C75D6B2834935B68C518BC04,B73B529994C4700157841432D224D947,1D81A638151A43E14D4F60CFAFF5E8E9,BCFD2A7425D57C797447D9E47A44A79F,ADDDD72469BDD7BD6B14BC21359C5CFF,3797D379D4D05BB733CA37EDFE74A29C,CE6E6B615231B052EF75AEDBED6491A5,325EAF17AD4FAE4A0B943066EF60BED1,D2C50D70FBCF0A56020B70626BF2FB2C,FF83472786DB3B25D48ABD61502E1D91,FF83472786DB3B25D48ABD61502E1D91,FF83472786DB3B25D48ABD61502E1D91,FF83472786DB3B25D48ABD61502E1D91,FF83472786DB3B25D48ABD61502E1D91,41B394758330C83757856AA482C79977
1,11317,10224,10198,113533,10125,9711,1,NaN,NaN,15Mar2018:14:08:14,01Jan5999:00:00:00,03Jan2020:07:01:13,A5C279811C3210EB8CFD7027E51ADAC4,1DA3C3531BF38D301A87CE1C24D69CCC,1B9D450C5EC57EF73BB95CF2F4A57F12,E5E1FA47D79C1D573A65E2A4D17F6E6E,3A73A94D029959DC71AF0AFCB9F2189E,0DC45A0D604E45ED8BA0A4ED142A386C,6EE6DFA5CFDC795C76C96862BC7F09A4,E5E1FA47D79C1D573A65E2A4D17F6E6E,BC8A69BA32EB41D5455D2EB96F8A12C6,713FF050F1DE07BB666862D98DB97371,E5E1FA47D79C1D573A65E2A4D17F6E6E,0281FFC7E864B8858E7611BB38B9B978,4CBE937C147AE54DF264D9443DBE7335,E5E1FA47D79C1D57

In [6]:
location_df = pd.read_csv("hse-dab-naf-23/LOCATION.csv", sep=';')

location_df.head(10)

,STORE_LOCATION_RK,STORE_LOCATION_LVL_RK4,STORE_LOCATION_LVL_RK3,STORE_LOCATION_LVL_RK2,STORE_LOCATION_LVL_RK1,STORE_OPEN_DTTM,STORE_CLOSURE_DTTM,VALID_FROM_DTTM,VALID_TO_DTTM,PROCESSED_DTTM,STORE_LOCATION_ADK_hashing,STORE_LOCATION_ID_hashing,STORE_LOCATION_NM_hashing,STORE_LOCATION_DESC_hashing,LANGUAGE_CD_hashing,STORE_LOCATION_LVL_ID4_hashing,STORE_LOCATION_LVL_NM4_hashing,STORE_LOCATION_LVL_DESC4_hashing,STORE_LOCATION_LVL_ID3_hashing,STORE_LOCATION_LVL_NM3_hashing,STORE_LOCATION_LVL_DESC3_hashing,STORE_LOCATION_LVL_ID2_hashing,STORE_LOCATION_LVL_NM2_hashing,STORE_LOCATION_LVL_DESC2_hashing,STORE_LOCATION_LVL_ID1_hashing,STORE_LOCATION_LVL_NM1_hashing,STORE_LOCATION_LVL_DESC1_hashing,STORE_LOCATION_ATTRIB1_hashing,STORE_LOCATION_ATTRIB2_hashing,STORE_LOCATION_ATTRIB3_hashing,STORE_LOCATION_ATTRIB4_hashing,STORE_LOCATION_ATTRIB5_hashing,STORE_LOCATION_ATTRIB6_hashing,STORE_LOCATION_ATTRIB7_hashing,STORE_LOCATION_ATTRIB8_hashing,STORE_LOCATION_ATTRIB9_hashing,STORE_LOCATION_ATTRIB10_hashing,STORE_LOCATION_ATTRIB11_hashing,STORE_LOCATION_ATTRIB12_hashing,STORE_LOCATION_ATTRIB13_hashing,STORE_LOCATION_ATTRIB14_hashing,STORE_LOCATION_ATTRIB15_hashing,STORE_LOCATION_ATTRIB16_hashing,STORE_LOCATION_ATTRIB17_hashing,STORE_LOCATION_ATTRIB18_hashing,STORE_LOCATION_ATTRIB19_hashing,STORE_LOCATION_ATTRIB20_hashing,STORE_LOCATION_ATTRIB21_hashing
0,277,124,124,19,1,NaN,NaN,15Mar2018:14:08:08,01Jan5999:00:00:00,03Jan2020:07:00:25,181F921CC957DF5CEC4BF2C18E4A371F,70BAE846650DAC2A5E3F42EE589903CB,1FD625282F5BB57C4CEF60EC78AB2BD5,E5E1FA47D79C1D573A65E2A4D17F6E6E,3A73A94D029959DC71AF0AFCB9F2189E,8786564D12D649C25FE5996C720486A4,691ED43F4AF65A71A6FB21210CBE5109,E5E1FA47D79C1D573A65E2A4D17F6E6E,8786564D12D649C25FE5996C720486A4,691ED43F4AF65A71A6FB21210CBE5109,E5E1FA47D79C1D573A65E2A4D17F6E6E,22B68008A13B4124563213F9AB4D6E6C,0D630DC55688E98737D478D34874D724,E5E1FA47D79C1D573A65E2A4D17F6E6E,B25D27739F2C724A6460B006215CCF2D,7120DC1E45C965E3B794F00665BDBC24,E5E1FA47D79C1D573A65E2A4D17F6E6E,B4E5DABD7AF605C64E1CEF8BE96EDEF0,1B208496FEF79DB5BE9F374BF65865E6,9B9B954D2D5B0F1C5B0B129DF8DED631,691ED43F4AF65A71A6FB21210CBE5109,2356B83B3AE123A290CB245A8F8526DA,75300434BE16A57FD386B22D19B289D5,CB889E260FC7B06777314137C2FB5198,1FAA6C1A11273624AB0AB65A2B4088E8,D2C50D70FBCF0A56020B70626BF2FB2C,410E87854BCC80640DBEFAB137107251,A067F6FD038AB399337304ED26250C88,6BA1D912365C006B003750C17E810A8C,6BA1D912365C006B003750C17E810A8C,B70EA5F8E1DD1E21F7485AADD433620F,FC5FD090934CE685C1C12E5E0311F1D4,3F0DA22658EF921D0F500BFDC5471DB3,FF83472786DB3B25D48ABD61502E1D91,FF83472786DB3B25D48ABD61502E1D91,FF83472786DB3B25D48ABD61502E1D91,FF83472786DB3B25D48ABD61502E1D91,FF83472786DB3B25D48ABD61502E1D91
1,278,124,124,19,1,NaN,NaN,15Mar2018:14:08:08,01Jan5999:00:00:00,03Jan2020:07:00:25,181F921CC957DF5CEC4BF2C18E4A371F,FDD55E57DC6CA449FFBB53DFC5960C74,7A390B982C99DE39EB8BDE09448B1B56,E5E1FA47D79C1D573A65E2A4D17F6E6E,3A73A94D029959DC71AF0AFCB9F2189E,8786564D12D649C25FE5996C720486A4,691ED43F4AF65A71A6FB21210CBE5109,E5E1FA47D79C1D573A65E2A4D17F6E6E,8786564D12D649C25FE5996C720486A4,691ED43F4AF65A71A6FB21210CBE5109,E5E1FA47D79C1D573A65E2A4D17F6E6E,22B68008A13B4124563213F9AB4D6E6C,0D630DC55688E98737D478D34874D724,E5E1FA47D79C1D573A65E2A4D17F6E6E,B25D27739F2C724A6460B006215CCF2D,7120DC1E45C965E3B794F00665BDBC24,E5E1FA47D79C1D573A65E2A4D17F6E6E,B4E5DABD7AF605C64E1CEF8BE96EDEF0,1B208496FEF79DB5BE9F374BF65865E6,81C7FB4E26CB45F0745DF4A59EB6CD9E,691ED43F4AF65A71A6FB21210CBE5109,2356B83B3AE123A290CB245A8F8526DA,75300434BE16A57FD386B22D19B289D5,CB889E260FC7B06777314137C2FB5198,1FAA6C1A11273624AB0AB65A2B4088E8,D2C50D70FBCF0A56020B70626BF2FB2C,410E87854BCC80640DBEFAB137107251,A067F6FD038AB399337304ED26250C88,6BA1D912365C006B003750C17E810A8C,6BA1D912365C006B003750C17E810A8C,B70EA5F8E1DD1E21F7485AADD433620F,0A7D98B3F3BA0A5BC89F693FB146FE40,3F0DA22658EF921D0F500BFDC5471DB3,FF83472786DB3B25D48ABD61502E1D91,FF83472786DB3B25D48ABD61502E1D91,FF83472786DB3B25D48ABD61502E1D91,FF83472786DB3B25D48ABD61502E1